In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# 🤖 Proactive Business Analytics & Anomaly Detection Agent System\n",
        "## Multi-Agent MVP - Kaggle Capstone Demo\n",
        "\n",
        "**Kaggle Agents Intensive - Final Project**\n",
        "\n",
        "This notebook demonstrates a multi-agent system that automatically analyzes business operations, detects anomalies, and provides actionable recommendations.\n",
        "\n",
        "### Key Components:\n",
        "- 🔄 **Coordinator Agent** — Orchestrates all analysis\n",
        "- 📊 **Historical Analyzer** — Detects past anomalies\n",
        "- 📡 **Live Monitor** — Real-time metric alerts\n",
        "- 💡 **Recommender** — Actionable recommendations\n",
        "\n",
        "---"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Cell 1: Setup & Import\n",
        "\n",
        "Install dependencies and import required modules."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Install dependencies\n",
        "!pip install google-generativeai pydantic pyyaml pandas numpy pytest -q\n",
        "\n",
        "import asyncio\n",
        "import json\n",
        "from datetime import datetime\n",
        "from dataclasses import dataclass, field\n",
        "from typing import Dict, List, Any, Optional\n",
        "import uuid\n",
        "import time\n",
        "\n",
        "print(\"✅ Dependencies installed\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Cell 2: Define Core Classes\n",
        "\n",
        "Define AgentConfig, AgentResponse, and AgentBase for the agent framework."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# === BASE AGENT CLASSES ===\n",
        "\n",
        "@dataclass\n",
        "class AgentConfig:\n",
        "    \"\"\"Agent configuration.\"\"\"\n",
        "    name: str\n",
        "    agent_type: str  # \"coordinator\", \"analyzer\", \"monitor\", \"recommender\"\n",
        "    model: str = \"gemini-2.0-flash\"\n",
        "    temperature: float = 0.7\n",
        "    max_tokens: int = 2048\n",
        "    system_prompt: Optional[str] = None\n",
        "    tools: List[str] = field(default_factory=list)\n",
        "    \n",
        "    def to_dict(self) -> Dict:\n",
        "        return {\n",
        "            \"name\": self.name,\n",
        "            \"agent_type\": self.agent_type,\n",
        "            \"model\": self.model,\n",
        "            \"temperature\": self.temperature,\n",
        "        }\n",
        "\n",
        "\n",
        "@dataclass\n",
        "class AgentResponse:\n",
        "    \"\"\"Standardized agent output.\"\"\"\n",
        "    agent_name: str\n",
        "    status: str  # \"success\", \"error\", \"partial\"\n",
        "    result: Any\n",
        "    reasoning: str = \"\"\n",
        "    tools_used: List[str] = field(default_factory=list)\n",
        "    execution_time_ms: float = 0.0\n",
        "    timestamp: str = field(default_factory=lambda: datetime.now().isoformat())\n",
        "    trace_id: str = field(default_factory=lambda: str(uuid.uuid4()))\n",
        "    metadata: Dict = field(default_factory=dict)\n",
        "    \n",
        "    def to_dict(self) -> Dict:\n",
        "        return {\n",
        "            \"agent_name\": self.agent_name,\n",
        "            \"status\": self.status,\n",
        "            \"result\": self.result,\n",
        "            \"reasoning\": self.reasoning,\n",
        "            \"tools_used\": self.tools_used,\n",
        "            \"execution_time_ms\": self.execution_time_ms,\n",
        "            \"timestamp\": self.timestamp,\n",
        "            \"trace_id\": self.trace_id,\n",
        "        }\n",
        "\n",
        "\n",
        "class AgentBase:\n",
        "    \"\"\"Base class for all agents.\"\"\"\n",
        "    \n",
        "    def __init__(self, config: AgentConfig):\n",
        "        self.config = config\n",
        "        self.logger = None\n",
        "        self.tools_registry = {}\n",
        "        \n",
        "    def log(self, level: str, message: str, extra: Dict = None):\n",
        "        \"\"\"Log agent action.\"\"\"\n",
        "        if self.logger:\n",
        "            self.logger(level, f\"[{self.config.name}] {message}\", extra or {})\n",
        "        else:\n",
        "            print(f\"[{level}] [{self.config.name}] {message}\")\n",
        "    \n",
        "    async def execute(self, task: str, context: Dict = None) -> AgentResponse:\n",
        "        \"\"\"Execute agent task.\"\"\"\n",
        "        raise NotImplementedError(\"Subclasses must override execute()\")\n",
        "\n",
        "print(\"✅ Base agent classes defined\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Cell 3: Define Mock Sub-Agents\n",
        "\n",
        "Create mock implementations of Historical Analyzer, Live Monitor, and Recommender."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# === MOCK SUB-AGENTS ===\n",
        "\n",
        "class HistoricalAnalyzerAgent(AgentBase):\n",
        "    \"\"\"Analyzes historical trends and detects anomalies.\"\"\"\n",
        "    \n",
        "    async def execute(self, task: str, context: Dict = None) -> AgentResponse:\n",
        "        start_time = time.time()\n",
        "        self.log(\"INFO\", \"🔍 Analyzing historical data...\")\n",
        "        await asyncio.sleep(0.5)  # Simulate work\n",
        "        \n",
        "        result_data = {\n",
        "            \"anomalies\": [\n",
        "                \"Sales drop by 34% on 2024-11-15\",\n",
        "                \"Unusual cost spike (+46.7%) on 2024-11-10\"\n",
        "            ],\n",
        "            \"severity\": [\"high\", \"medium\"],\n",
        "            \"detailed_findings\": [\n",
        "                {\"metric\": \"Revenue\", \"deviation\": -34.1, \"date\": \"2024-11-15\"},\n",
        "                {\"metric\": \"Costs\", \"deviation\": 46.7, \"date\": \"2024-11-10\"}\n",
        "            ]\n",
        "        }\n",
        "        \n",
        "        return AgentResponse(\n",
        "            agent_name=self.config.name,\n",
        "            status=\"success\",\n",
        "            result=result_data,\n",
        "            reasoning=\"Statistical anomaly detection (IQR method) identified 2 significant deviations\",\n",
        "            tools_used=[\"detect_anomalies\", \"analyze_trends\"],\n",
        "            execution_time_ms=(time.time() - start_time) * 1000\n",
        "        )\n",
        "\n",
        "\n",
        "class LiveMonitorAgent(AgentBase):\n",
        "    \"\"\"Monitors real-time metrics and generates alerts.\"\"\"\n",
        "    \n",
        "    async def execute(self, task: str, context: Dict = None) -> AgentResponse:\n",
        "        start_time = time.time()\n",
        "        self.log(\"INFO\", \"📡 Monitoring live metrics...\")\n",
        "        await asyncio.sleep(0.4)  # Simulate work\n",
        "        \n",
        "        result_data = {\n",
        "            \"alerts\": [\n",
        "                \"Customer churn rate up 5% vs baseline\",\n",
        "                \"Inventory below safety threshold (120 units)\"\n",
        "            ],\n",
        "            \"alert_severity\": [\"high\", \"medium\"],\n",
        "            \"alert_count\": 2\n",
        "        }\n",
        "        \n",
        "        return AgentResponse(\n",
        "            agent_name=self.config.name,\n",
        "            status=\"success\",\n",
        "            result=result_data,\n",
        "            reasoning=\"Real-time threshold analysis detected 2 metric deviations\",\n",
        "            tools_used=[\"monitor_metrics\", \"threshold_check\"],\n",
        "            execution_time_ms=(time.time() - start_time) * 1000\n",
        "        )\n",
        "\n",
        "\n",
        "class RecommenderAgent(AgentBase):\n",
        "    \"\"\"Generates actionable recommendations.\"\"\"\n",
        "    \n",
        "    async def execute(self, task: str, context: Dict = None) -> AgentResponse:\n",
        "        start_time = time.time()\n",
        "        self.log(\"INFO\", \"💡 Generating recommendations...\")\n",
        "        await asyncio.sleep(0.5)  # Simulate work\n",
        "        \n",
        "        result_data = {\n",
        "            \"recommendations\": [\n",
        "                {\n",
        "                    \"priority\": 1,\n",
        "                    \"issue\": \"Sales drop on 2024-11-15\",\n",
        "                    \"action\": \"Investigate customer complaints and market competitor activity\",\n",
        "                    \"estimated_impact\": \"High ($23k+ recovery potential)\",\n",
        "                    \"timeline\": \"Immediate (24 hours)\"\n",
        "                },\n",
        "                {\n",
        "                    \"priority\": 2,\n",
        "                    \"issue\": \"Customer churn rate increase\",\n",
        "                    \"action\": \"Launch customer retention campaign and review satisfaction surveys\",\n",
        "                    \"estimated_impact\": \"High (preserve recurring revenue)\",\n",
        "                    \"timeline\": \"This week\"\n",
        "                }\n",
        "            ],\n",
        "            \"summary\": \"Generated 2 prioritized recommendations\"\n",
        "        }\n",
        "        \n",
        "        return AgentResponse(\n",
        "            agent_name=self.config.name,\n",
        "            status=\"success\",\n",
        "            result=result_data,\n",
        "            reasoning=\"Analyzed all findings and generated prioritized, actionable recommendations\",\n",
        "            tools_used=[\"reason_about_causes\", \"generate_recommendations\", \"prioritize_actions\"],\n",
        "            execution_time_ms=(time.time() - start_time) * 1000\n",
        "        )\n",
        "\n",
        "print(\"✅ Mock sub-agents defined\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Cell 4: Define Coordinator Agent\n",
        "\n",
        "Create the main Coordinator that orchestrates all sub-agents."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# === COORDINATOR AGENT ===\n",
        "\n",
        "class CoordinatorAgent(AgentBase):\n",
        "    \"\"\"Orchestrates all sub-agents.\"\"\"\n",
        "    \n",
        "    def __init__(self, config: AgentConfig):\n",
        "        super().__init__(config)\n",
        "        self.sub_agents: Dict[str, AgentBase] = {}\n",
        "    \n",
        "    def register_sub_agent(self, name: str, agent: AgentBase):\n",
        "        \"\"\"Register a sub-agent.\"\"\"\n",
        "        self.sub_agents[name] = agent\n",
        "        self.log(\"INFO\", f\"Registered sub-agent: {name}\")\n",
        "    \n",
        "    async def execute(self, task: str, context: Dict = None) -> AgentResponse:\n",
        "        \"\"\"Execute coordinated analysis.\"\"\"\n",
        "        start_time = time.time()\n",
        "        all_results = {}\n",
        "        reasoning_steps = []\n",
        "        \n",
        "        try:\n",
        "            self.log(\"INFO\", \"🔄 Starting coordinated analysis\")\n",
        "            reasoning_steps.append(\"Analysis initiated\")\n",
        "            \n",
        "            # Call Historical Analyzer\n",
        "            self.log(\"INFO\", \"📊 Calling Historical Analyzer...\")\n",
        "            if \"historical_analyzer\" in self.sub_agents:\n",
        "                historical_result = await self.sub_agents[\"historical_analyzer\"].execute(\n",
        "                    \"Analyze trends\", context\n",
        "                )\n",
        "                all_results[\"historical\"] = historical_result\n",
        "                reasoning_steps.append(\"✓ Historical analysis complete\")\n",
        "            \n",
        "            # Call Live Monitor\n",
        "            self.log(\"INFO\", \"📡 Calling Live Monitor...\")\n",
        "            if \"live_monitor\" in self.sub_agents:\n",
        "                live_result = await self.sub_agents[\"live_monitor\"].execute(\n",
        "                    \"Monitor metrics\", context\n",
        "                )\n",
        "                all_results[\"live\"] = live_result\n",
        "                reasoning_steps.append(\"✓ Live monitoring complete\")\n",
        "            \n",
        "            # Call Recommender\n",
        "            self.log(\"INFO\", \"💡 Calling Recommender...\")\n",
        "            if \"recommender\" in self.sub_agents:\n",
        "                rec_result = await self.sub_agents[\"recommender\"].execute(\n",
        "                    \"Generate recommendations\", context\n",
        "                )\n",
        "                all_results[\"recommendations\"] = rec_result\n",
        "                reasoning_steps.append(\"✓ Recommendations generated\")\n",
        "            \n",
        "            # Synthesize final result\n",
        "            final_result = self._synthesize_report(all_results)\n",
        "            \n",
        "            execution_time = (time.time() - start_time) * 1000\n",
        "            \n",
        "            return AgentResponse(\n",
        "                agent_name=self.config.name,\n",
        "                status=\"success\",\n",
        "                result=final_result,\n",
        "                reasoning=\"\\n\".join(reasoning_steps),\n",
        "                tools_used=list(self.sub_agents.keys()),\n",
        "                execution_time_ms=execution_time\n",
        "            )\n",
        "        \n",
        "        except Exception as e:\n",
        "            self.log(\"ERROR\", f\"Error: {str(e)}\")\n",
        "            return AgentResponse(\n",
        "                agent_name=self.config.name,\n",
        "                status=\"error\",\n",
        "                result={\"error\": str(e)},\n",
        "                execution_time_ms=(time.time() - start_time) * 1000\n",
        "            )\n",
        "    \n",
        "    def _synthesize_report(self, all_results: Dict) -> Dict:\n",
        "        \"\"\"Synthesize final report from all results.\"\"\"\n",
        "        issues = []\n",
        "        \n",
        "        # Collect issues from historical analysis\n",
        "        if \"historical\" in all_results:\n",
        "            hist = all_results[\"historical\"].result\n",
        "            for i, anomaly in enumerate(hist.get(\"anomalies\", [])):\n",
        "                severity = hist.get(\"severity\", [\"medium\"] * len(hist.get(\"anomalies\", [])))[i]\n",
        "                issues.append({\n",
        "                    \"id\": f\"hist_{i}\",\n",
        "                    \"source\": \"Historical Analysis\",\n",
        "                    \"description\": anomaly,\n",
        "                    \"severity\": severity\n",
        "                })\n",
        "        \n",
        "        # Collect alerts from live monitor\n",
        "        if \"live\" in all_results:\n",
        "            live = all_results[\"live\"].result\n",
        "            for i, alert in enumerate(live.get(\"alerts\", [])):\n",
        "                severity = live.get(\"alert_severity\", [\"medium\"] * len(live.get(\"alerts\", [])))[i]\n",
        "                issues.append({\n",
        "                    \"id\": f\"live_{i}\",\n",
        "                    \"source\": \"Live Monitor\",\n",
        "                    \"description\": alert,\n",
        "                    \"severity\": severity\n",
        "                })\n",
        "        \n",
        "        # Rank by severity\n",
        "        severity_rank = {\"critical\": 0, \"high\": 1, \"medium\": 2, \"low\": 3}\n",
        "        issues_ranked = sorted(issues, key=lambda x: severity_rank.get(x[\"severity\"], 999))\n",
        "        \n",
        "        recommendations = []\n",
        "        if \"recommendations\" in all_results:\n",
        "            recommendations = all_results[\"recommendations\"].result.get(\"recommendations\", [])\n",
        "        \n",
        "        return {\n",
        "            \"analysis_completed_at\": datetime.now().isoformat(),\n",
        "            \"total_issues_found\": len(issues),\n",
        "            \"issues_by_severity\": {\n",
        "                \"critical\": sum(1 for i in issues if i[\"severity\"] == \"critical\"),\n",
        "                \"high\": sum(1 for i in issues if i[\"severity\"] == \"high\"),\n",
        "                \"medium\": sum(1 for i in issues if i[\"severity\"] == \"medium\"),\n",
        "                \"low\": sum(1 for i in issues if i[\"severity\"] == \"low\"),\n",
        "            },\n",
        "            \"ranked_issues\": issues_ranked,\n",
        "            \"recommendations\": recommendations,\n",
        "            \"next_actions\": [\n",
        "                \"Review top-priority issues immediately\",\n",
        "                \"Implement recommended actions\",\n",
        "                \"Monitor progress and feedback\"\n",
        "            ]\n",
        "        }\n",
        "\n",
        "print(\"✅ Coordinator Agent defined\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Cell 5: Run End-to-End Demo\n",
        "\n",
        "Execute the full multi-agent analysis with mock data."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# === RUN DEMO ===\n",
        "\n",
        "async def run_analysis():\n",
        "    \"\"\"Run the full multi-agent analysis.\"\"\"\n",
        "    \n",
        "    print(\"\\n\" + \"=\"*70)\n",
        "    print(\"🤖 MULTI-AGENT BUSINESS ANALYTICS SYSTEM (MVP DEMO)\")\n",
        "    print(\"=\"*70 + \"\\n\")\n",
        "    \n",
        "    # Create Coordinator\n",
        "    coord_config = AgentConfig(\n",
        "        name=\"Coordinator\",\n",
        "        agent_type=\"coordinator\",\n",
        "        temperature=0.5\n",
        "    )\n",
        "    coordinator = CoordinatorAgent(coord_config)\n",
        "    \n",
        "    # Register sub-agents\n",
        "    print(\"📋 Registering agents...\")\n",
        "    coordinator.register_sub_agent(\n",
        "        \"historical_analyzer\",\n",
        "        HistoricalAnalyzerAgent(AgentConfig(name=\"HistoricalAnalyzer\", agent_type=\"analyzer\"))\n",
        "    )\n",
        "    coordinator.register_sub_agent(\n",
        "        \"live_monitor\",\n",
        "        LiveMonitorAgent(AgentConfig(name=\"LiveMonitor\", agent_type=\"monitor\"))\n",
        "    )\n",
        "    coordinator.register_sub_agent(\n",
        "        \"recommender\",\n",
        "        RecommenderAgent(AgentConfig(name=\"Recommender\", agent_type=\"recommender\"))\n",
        "    )\n",
        "    \n",
        "    # Prepare context\n",
        "    context = {\n",
        "        \"company_name\": \"Acme Corp\",\n",
        "        \"analysis_period\": \"Q4 2024\",\n",
        "        \"metrics_file\": \"sample_business_metrics.csv\"\n",
        "    }\n",
        "    \n",
        "    # Execute analysis\n",
        "    print(\"\\n🔄 Starting coordinated analysis...\\n\")\n",
        "    response = await coordinator.execute(\n",
        "        \"Analyze Q4 business performance and identify bottlenecks\",\n",
        "        context\n",
        "    )\n",
        "    \n",
        "    # Display results\n",
        "    print(\"\\n\" + \"=\"*70)\n",
        "    print(\"📊 ANALYSIS RESULTS\")\n",
        "    print(\"=\"*70 + \"\\n\")\n",
        "    \n",
        "    if response.status == \"success\":\n",
        "        result = response.result\n",
        "        \n",
        "        print(f\"✅ Status: {response.status.upper()}\")\n",
        "        print(f\"⏱️  Execution Time: {response.execution_time_ms:.1f}ms\")\n",
        "        \n",
        "        print(f\"\\n📊 Summary:\")\n",
        "        print(f\"  Total Issues Found: {result['total_issues_found']}\")\n",
        "        print(f\"  - Critical: {result['issues_by_severity']['critical']}\")\n",
        "        print(f\"  - High: {result['issues_by_severity']['high']}\")\n",
        "        print(f\"  - Medium: {result['issues_by_severity']['medium']}\")\n",
        "        print(f\"  - Low: {result['issues_by_severity']['low']}\")\n",
        "        \n",
        "        print(f\"\\n🎯 Ranked Issues (by Severity):\")\n",
        "        for i, issue in enumerate(result['ranked_issues'], 1):\n",
        "            print(f\"  {i}. [{issue['severity'].upper()}] {issue['description']}\")\n",
        "            print(f\"     Source: {issue['source']}\")\n",
        "        \n",
        "        print(f\"\\n💡 Recommendations:\")\n",
        "        for i, rec in enumerate(result['recommendations'], 1):\n",
        "            print(f\"  {i}. [Priority {rec['priority']}] {rec['issue']}\")\n",
        "            print(f\"     Action: {rec['action']}\")\n",
        "            print(f\"     Impact: {rec['estimated_impact']}\")\n",
        "            print(f\"     Timeline: {rec['timeline']}\")\n",
        "        \n",
        "        print(f\"\\n📋 Next Steps:\")\n",
        "        for i, action in enumerate(result['next_actions'], 1):\n",
        "            print(f\"  {i}. {action}\")\n",
        "    \n",
        "    else:\n",
        "        print(f\"❌ Status: {response.status.upper()}\")\n",
        "        print(f\"Error: {response.result.get('error', 'Unknown error')}\")\n",
        "    \n",
        "    print(\"\\n\" + \"=\"*70 + \"\\n\")\n",
        "    \n",
        "    return response\n",
        "\n",
        "# Run the demo\n",
        "response = await run_analysis()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Cell 6: Detailed Trace & Metrics\n",
        "\n",
        "View detailed execution traces and performance metrics."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# === DETAILED METRICS & TRACE ===\n",
        "\n",
        "print(\"📊 DETAILED EXECUTION TRACE\\n\")\n",
        "print(f\"Trace ID: {response.trace_id}\")\n",
        "print(f\"Timestamp: {response.timestamp}\")\n",
        "print(f\"Execution Time: {response.execution_time_ms:.1f}ms\")\n",
        "print(f\"\\nReasoning Steps:\")\n",
        "for step in response.reasoning.split(\"\\n\"):\n",
        "    if step:\n",
        "        print(f\"  - {step}\")\n",
        "\n",
        "print(f\"\\nTools Used:\")\n",
        "for tool in response.tools_used:\n",
        "    print(f\"  - {tool}\")\n",
        "\n",
        "print(f\"\\n📈 Performance Metrics:\")\n",
        "print(f\"  Total Agents: {len(response.tools_used)}\")\n",
        "print(f\"  Issues Detected: {response.result['total_issues_found']}\")\n",
        "print(f\"  Recommendations Generated: {len(response.result['recommendations'])}\")\n",
        "print(f\"  Average Issue Severity: {['high', 'medium'][response.result['issues_by_severity']['high'] <= response.result['issues_by_severity']['medium']]}\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Cell 7: Session Persistence (Optional)\n",
        "\n",
        "Demonstrate session storage and recovery capability."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# === SESSION PERSISTENCE DEMO ===\n",
        "\n",
        "import os\n",
        "\n",
        "# Create sessions directory\n",
        "os.makedirs(\"./sessions\", exist_ok=True)\n",
        "\n",
        "# Save session example\n",
        "session_data = {\n",
        "    \"session_id\": f\"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}_demo\",\n",
        "    \"created_at\": datetime.now().isoformat(),\n",
        "    \"company_name\": \"Acme Corp\",\n",
        "    \"analysis_period\": \"Q4 2024\",\n",
        "    \"issues_found\": response.result['total_issues_found'],\n",
        "    \"recommendations_count\": len(response.result['recommendations']),\n",
        "    \"status\": \"completed\"\n",
        "}\n",
        "\n",
        "session_file = f\"./sessions/{session_data['session_id']}.json\"\n",
        "with open(session_file, \"w\") as f:\n",
        "    json.dump(session_data, f, indent=2)\n",
        "\n",
        "print(\"💾 Session Management Demo\\n\")\n",
        "print(f\"✅ Session saved: {session_data['session_id']}\")\n",
        "print(f\"   Location: {session_file}\")\n",
        "print(f\"\\nSession Data:\")\n",
        "for key, value in session_data.items():\n",
        "    print(f\"  {key}: {value}\")\n",
        "\n",
        "# List all saved sessions\n",
        "sessions = [f[:-5] for f in os.listdir(\"./sessions\") if f.endswith(\".json\")]\n",
        "print(f\"\\nTotal saved sessions: {len(sessions)}\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Cell 8: Export Results\n",
        "\n",
        "Export analysis results and metrics for further analysis."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# === EXPORT RESULTS ===\n",
        "\n",
        "os.makedirs(\"./evaluation\", exist_ok=True)\n",
        "\n",
        "# Export full response\n",
        "response_file = f\"./evaluation/analysis_result_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json\"\n",
        "with open(response_file, \"w\") as f:\n",
        "    # Convert to dict for JSON serialization\n",
        "    response_dict = {\n",
        "        \"agent_name\": response.agent_name,\n",
        "        \"status\": response.status,\n",
        "        \"result\": response.result,\n",
        "        \"reasoning\": response.reasoning,\n",
        "        \"execution_time_ms\": response.execution_time_ms,\n",
        "        \"timestamp\": response.timestamp,\n",
        "        \"trace_id\": response.trace_id\n",
        "    }\n",
        "    json.dump(response_dict, f, indent=2, ensure_ascii=False)\n",
        "\n",
        "print(\"📤 EXPORT SUMMARY\\n\")\n",
        "print(f\"✅ Analysis results exported to: {response_file}\")\n",
        "print(f\"\\nFile contains:\")\n",
        "print(f\"  - Execution status and timing\")\n",
        "print(f\"  - Ranked issues by severity\")\n",
        "print(f\"  - Actionable recommendations\")\n",
        "print(f\"  - Agent reasoning and trace ID\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Summary\n",
        "\n",
        "### ✅ MVP Features Demonstrated:\n",
        "\n",
        "1. **Coordinator Agent** ✅\n",
        "   - Orchestrates 3 sub-agents sequentially\n",
        "   - Distributes tasks and collects results\n",
        "   - Synthesizes final ranked report\n",
        "\n",
        "2. **Historical Analyzer** ✅\n",
        "   - Detects anomalies in historical data\n",
        "   - Returns detailed findings\n",
        "\n",
        "3. **Live Monitor** ✅\n",
        "   - Monitors real-time metrics\n",
        "   - Generates real-time alerts\n",
        "\n",
        "4. **Recommender** ✅\n",
        "   - Generates actionable recommendations\n",
        "   - Prioritizes by impact and urgency\n",
        "\n",
        "5. **Session Management** ✅\n",
        "   - Saves and retrieves sessions\n",
        "   - Persistence to disk\n",
        "\n",
        "6. **Structured Output** ✅\n",
        "   - Ranked issues by severity\n",
        "   - Detailed recommendations\n",
        "   - Next action items\n",
        "\n",
        "---\n",
        "\n",
        "### 🚀 Next Steps:\n",
        "- **Day 2:** Replace mock agents with real tool implementations\n",
        "- **Day 3:** Integrate Gemini API for real agent reasoning\n",
        "- **Day 4:** Add advanced evaluation and observability\n",
        "- **Day 5+:** Production deployment and scaling\n",
        "\n",
        "---\n",
        "\n",
        "**Status:** ✅ MVP Complete - Ready for Tool Integration  \n",
        "**Kaggle Capstone Track:** Enterprise Agents  \n",
        "**Created:** 2024-11-21"
      ]
    }
  ],
  "metadata": {
    "kernelspec": {
      "display_name": "Python 3\",
      "language": "python\",
      "name": "python3"
    },
    "language_info": {
      "codemirror_mode": {
        \"name": "ipython\",
        \"version\": 3
      },
      "file_extension\": \".py\",
      \"mimetype\": \"text/x-python\",
      \"name\": \"python\",
      \"nbconvert_exporter\": \"python\",
      \"pygments_lexer\": \"ipython3\",
      \"version\": \"3.10.0\"
    }
  },
  "nbformat\": 4,
  "nbformat_minor\": 4
}
